In [60]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmaps
from matplotlib.colors import LogNorm

In [61]:
def worldmap(img, vmin=0, vmax=1e2):
    projection = ccrs.PlateCarree(central_longitude=0)
    img_extent = (-180, 180, -90, 90)

    fig = plt.figure()
    ax = plt.subplot(projection=projection)
    ax.coastlines()
    im = ax.imshow(img,
                   origin='upper',
                   extent=img_extent,
                   transform=projection,
                   cmap='GnBu',
                   vmin=vmin, vmax=vmax)
    fig.colorbar(im, ax=ax, orientation='horizontal')

In [62]:
def worldmap_norm(data, vmin=1e-30):
    projection = ccrs.PlateCarree(central_longitude=0)
    img_extent = (-180, 180, -90, 90)
    
    log_norm = LogNorm(vmin=vmin, vmax=data.max())
    #log_norm_data = log_norm(data, clip=True)

    fig = plt.figure()
    ax = plt.subplot(projection=projection)
    ax.coastlines()
    im = ax.imshow(data,
                   origin='upper',
                   extent=img_extent,
                   transform=projection,
                   cmap='GnBu',
                   norm=log_norm)
    fig.colorbar(im, ax=ax, orientation='horizontal')

In [63]:
def load(name, SUF='.hlf', day=356, second=86400, coef=1e12):
    if SUF == '.hlf':
        shape = (360, 720)
        areapath = '/home/kajiyama/H08/H08_20230612/map/dat/lnd_ara_/lndara.WFDEI.hlf'
        maskpath = '/home/kajiyama/H08/H08_20230612/map/dat/lnd_msk_/lndmsk.WFDEI.hlf'
    elif SUF == '.gl5':
        shape = (2160, 4320)
        areapath = '/home/kajiyama/H08/H08_20230612/map/dat/lnd_ara_/lndara.CAMA.gl5'
        maskpath = '/home/kajiyama/H08/H08_20230612/map/dat/lnd_msk_/lndmsk.CAMA.gl5'
    dtype = 'float32' # 4バイト
    lnddir = '/home/kajiyama/H08/H08_20230612/'
    file = lnddir + name
    
    # data
    data = np.fromfile(file, dtype=dtype)
    lonlat = data.reshape(shape)
    
    # mask out
    mask = np.fromfile(maskpath, dtype=dtype)
    mask = mask.reshape(shape)
    lonlat = np.ma.masked_where(mask==0, lonlat)
    lonlat = np.ma.masked_where(lonlat>1e19, lonlat)
    
    return lonlat

# RESULTS

In [65]:
ara = load(name='map/dat/lnd_ara_/lndara.CAMA.gl5', SUF='.gl5')
irg_1 = load(name='map/out/irg_frcd/S05_____20000000.gl5', SUF='.gl5')
irg_2 = load(name='map/out/irg_frcs/S05_____20000000.gl5', SUF='.gl5')
irg_3 = load(name='map/out/rfd_frc_/S05_____20000000.gl5', SUF='.gl5')
irg_4 = load(name='map/out/non_frc_/S05_____20000000.gl5', SUF='.gl5')

# 1ha = 10,000m2
mult_1 = ara*irg_1
mult_2 = ara*irg_2
mult_3 = ara*irg_3
mult_4 = ara*irg_4

count_1 = np.sum(mult_1 < 0)
count_2 = np.sum(mult_2 < 0)
count_3 = np.sum(mult_3 < 0)
count_4 = np.sum(mult_4 < 0)
print(f" irg_1 less than 0 is {count_1}")
print(f" irg_2 less than 0 is {count_2}")
print(f" irg_3 less than 0 is {count_3}")
print(f" irg_4 less than 0 is {count_4}")

# sum
total_1 = np.sum(mult_1)
total_2 = np.sum(mult_2)
total_3 = np.sum(mult_3)
total_4 = np.sum(mult_4)
print(f"irg_1 {total_1/1e4/1e6} million ha")
print(f"irg_2 {total_2/1e4/1e6} million ha")
print(f"irg_3 {total_3/1e4/1e6} million ha")
print(f"irg_4 {total_4/1e4/1e6} million ha")

print(f"area_total {(total_1 + total_2 + total_3 + total_4)/1e10} million ha")
print(f"area_true {np.sum(ara)/1e10} million ha")

 irg_1 less than 0 is 0
 irg_2 less than 0 is 0
 irg_3 less than 0 is 0
 irg_4 less than 0 is 0
irg_1 61.435687731200005 million ha
irg_2 210.759909376 million ha
irg_3 1269.9161329663998 million ha
irg_4 12064.5874614272 million ha
area_total 13606.69966336 million ha
area_true 13606.7046965248 million ha
